#  Coleta Automática de Preços de Ações com Python

Este notebook coleta automaticamente os preços de ações da B3 utilizando o Yahoo Finance e armazena os dados em um arquivo CSV. O processo é contínuo e ocorre a cada 15 minutos, permitindo a construção de um histórico de preços ao longo do dia.

---

## Objetivo:

- Buscar os preços das ações (ex: PETR4.SA, VALE3.SA)
- Armazenar os dados com horário da coleta
- Repetir esse processo a cada 15 minutos automaticamente

## Etapa 1 – Bibliotecas utilizadas

| Biblioteca      | Função                                                                 |
|----------------|------------------------------------------------------------------------|
| `yfinance`      | Buscar preços das ações diretamente do Yahoo Finance                   |
| `pandas`        | Organizar os dados em tabelas e salvar em CSV                          |
| `schedule`      | Agendar tarefas automáticas de coleta a cada X minutos                 |
| `datetime`      | Gerar data e hora da coleta                                            |
| `time`          | Controlar o tempo de espera entre coletas                              |
| `os`            | Verificar se o arquivo já existe para atualizar sem sobrescrever       |

In [ ]:
!pip install schedule --quiet

In [ ]:
# Instale o yfinance caso ainda não tenha
!pip install yfinance --quiet

import yfinance as yf
import pandas as pd
import time
import schedule
import os
from datetime import datetime


## Etapa 2 – Configuração inicial

Definimos:
- Quais ações vamos monitorar
- O nome do arquivo onde os dados serão armazenados


In [ ]:
acoes = ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA']  # Ações que queremos monitorar
arquivo_saida = 'dados_acoes.csv'            # Nome do arquivo que guardará os dados

## Etapa 3 – Função de coleta de dados

Essa função:
- Coleta o preço mais recente de cada ação
- Registra o horário da coleta
- Salva ou atualiza o arquivo `dados_acoes.csv`


In [15]:
def coletar_dados():
    agora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    dados = []

    for acao in acoes:
        info = yf.Ticker(acao).history(period='1d', interval='1m')
        if not info.empty:
            preco_atual = info['Close'].iloc[-1]
            dados.append({'timestamp': agora, 'acao': acao, 'preco': preco_atual})

    df_novo = pd.DataFrame(dados)

    if os.path.exists(arquivo_saida):
        df_existente = pd.read_csv(arquivo_saida)
        df_total = pd.concat([df_existente, df_novo], ignore_index=True)
    else:
        df_total = df_novo

    df_total.to_csv(arquivo_saida, index=False)
    print(f"[{agora}] Dados coletados e salvos.")


## Etapa 4 – Agendamento da coleta

Vamos configurar para coletar dados a cada 15 minutos automaticamente.

In [16]:
schedule.every(15).minutes.do(coletar_dados)

print("⏳ Iniciando o monitoramento. Pressione STOP para encerrar a execução.")

while True:
    schedule.run_pending()
    time.sleep(60)

⏳ Iniciando o monitoramento. Pressione STOP para encerrar a execução.
[2025-06-08 15:19:06] Dados coletados e salvos.
[2025-06-08 15:20:07] Dados coletados e salvos.
[2025-06-08 15:21:08] Dados coletados e salvos.


KeyboardInterrupt: 

In [17]:
pd.read_csv('dados_acoes.csv')

,timestamp,acao,preco
0,2025-06-08 15:11:16,PETR4.SA,29.690001
1,2025-06-08 15:11:16,VALE3.SA,52.930000
2,2025-06-08 15:11:16,ITUB4.SA,36.520000
3,2025-06-08 15:12:39,PETR4.SA,29.690001
4,2025-06-08 15:12:39,VALE3.SA,52.930000
5,2025-06-08 15:12:39,ITUB4.SA,36.520000
6,2025-06-08 15:19:06,PETR4.SA,29.690001
7,2025-06-08 15:19:06,VALE3.SA,52.930000
8,2025-06-08 15:19:06,ITUB4.SA,36.520000
9,2025-06-08 15:20:07,PETR4.SA,29.690001


## Cookie Clicker Exemplo

Neste exemplo, eu tentei criar um programa que automatizava a produção de cookies do jogo Cookie Cliekers todavia, enfrentei problemas constantes de CAPTCHA (Completely Automated Public Turing test to tell Computers and Humans Apart)

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import random

# Configurações para "disfarçar" o Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--profile-directory=Default")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--incognito")

# Finge que é um navegador comum
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

# Caminho do ChromeDriver
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

# Acessa o Google
driver.get("https://www.google.com")

# Espera a barra de busca aparecer
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "q"))
)

# Digita "tech with tim" e pressiona ENTER
search_input = driver.find_element(By.NAME, "q")
search_input.clear()
search_input.send_keys("Pesquisa Automatizada" + Keys.ENTER)

# Espera o carregamento da página de resultados
time.sleep(random.uniform(3, 5))

# Fecha o navegador
driver.quit()


## Sobre o Código:

O `webdriver` é o componente do Selenium que permite controlar o navegador de forma programada. No nosso código, ele está sendo utilizado para abrir o Google Chrome, acessar o site google.com, digitar uma busca automaticamente e simular o comportamento de um usuário humano. Ele faz isso a partir da linha `driver = webdriver.Chrome(...)`, onde o navegador é iniciado com uma série de opções definidas anteriormente.

Essas opções são a chave para burlar o CAPTCHA do Google. Usamos argumentos como `--disable-blink-features=AutomationControlled` e um `user-agent` realista para evitar que o navegador revele que está sendo controlado por um script. Também abrimos o navegador em modo anônimo para simular melhor o comportamento humano. Com esses ajustes, conseguimos enganar os mecanismos de detecção automática do Google, que normalmente ativariam um CAPTCHA ao perceberem que a navegação está sendo feita por um robô.

Ao fim, o código simula o ato de pesquisar no Google por “Pesquisa Automatizada” e espera alguns segundos antes de encerrar o navegador. O segredo para evitar o CAPTCHA foi justamente "camuflar" a automação com configurações que fazem o navegador parecer legítimo e humano.

![Pesquisa_Automatizada](pesquisa_auto.png)


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# === User-Agent que você copiou do seu navegador ===
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

# === Cookie cf_clearance ===
COOKIES = [
    {
        "name": "cf_clearance",
        "value": "cbiV8HjN2qWQbySx3w3itHOslkOS7ga4Feh_Y.YDF2c-1749405576-1.2.1.1-qL0k6H1C5xr9.la33PwXMsYbCseBFOxvDKeWOUM7Gen0SUL5c19GW5Y0hUwwMxUCHKB_UzfZvy4.DowfxcgsSQeh3jmv9jKuAmXhcXl2JMyqYFTJlt7WLFObPlwEdkULvG5C9qGXUdVLTDjvujJuDljjPuq_k6YpraJ4rFxgWzBs_T7FW4Ri70TqwPtIoVr0po0Sf.5z7h5BiIHfcpH0U3aHSjErBDNSPFN3INRM8R70ljwbucLj5tDK5GMqO5CfYdpjV3M9zVF3BiN0U1KBRr8fg4BrIMS1AP2vu562aUELP7DhrreJm8.p_DJiAOGXXb7.aK2OLSVkK7oGXZhtMAFV8fZX6oSZnrKN00trrCY",
        "domain": ".dashnet.org",
        "path": "/",
        "secure": True,
        "httpOnly": True
    }
]

# === Configurações do navegador ===
chrome_options = Options()
chrome_options.add_argument(f"user-agent={USER_AGENT}")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--start-maximized")

# === Inicializa o Chrome ===
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

# === Primeiro acesso ao domínio (obrigatório para setar cookies) ===
driver.get("https://orteil.dashnet.org")

# === Adiciona o cookie manualmente ===
for cookie in COOKIES:
    driver.add_cookie(cookie)

# === Agora acessa o jogo com a sessão autenticada ===
driver.get("https://orteil.dashnet.org/cookieclicker")

# === Aguarda o botão de idioma aparecer e clica ===
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'English')]"))
)
driver.find_element(By.XPATH, "//*[contains(text(), 'English')]").click()

# === Aguarda o cookie aparecer e interage ===
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.ID, "bigCookie"))
)
cookie = driver.find_element(By.ID, "bigCookie")

# === Loop de cliques com compra automática básica ===
product_price_prefix = "productPrice"
product_prefix = "product"

while True:
    cookie.click()
    time.sleep(0.01)

    cookies_count = driver.find_element(By.ID, "cookies").text.split(" ")[0]
    cookies_count = int(cookies_count.replace(",", ""))

    for i in range(4):
        try:
            price_el = driver.find_element(By.ID, product_price_prefix + str(i)).text.replace(",", "")
            if not price_el.isdigit():
                continue
            price = int(price_el)

            if cookies_count >= price:
                driver.find_element(By.ID, product_prefix + str(i)).click()
                break
        except:
            continue


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.69)
Stacktrace:
	GetHandleVerifier [0x0x7ff7d550fea5+79173]
	GetHandleVerifier [0x0x7ff7d550ff00+79264]
	(No symbol) [0x0x7ff7d52c9e5a]
	(No symbol) [0x0x7ff7d52b5c25]
	(No symbol) [0x0x7ff7d52dac44]
	(No symbol) [0x0x7ff7d53503c5]
	(No symbol) [0x0x7ff7d5370922]
	(No symbol) [0x0x7ff7d5348743]
	(No symbol) [0x0x7ff7d53114c1]
	(No symbol) [0x0x7ff7d5312253]
	GetHandleVerifier [0x0x7ff7d57da2dd+3004797]
	GetHandleVerifier [0x0x7ff7d57d472d+2981325]
	GetHandleVerifier [0x0x7ff7d57f3380+3107360]
	GetHandleVerifier [0x0x7ff7d552aa2e+188622]
	GetHandleVerifier [0x0x7ff7d55322bf+219487]
	GetHandleVerifier [0x0x7ff7d5518df4+115860]
	GetHandleVerifier [0x0x7ff7d5518fa9+116297]
	GetHandleVerifier [0x0x7ff7d54ff558+11256]
	BaseThreadInitThunk [0x0x7ffda1ffe8d7+23]
	RtlUserThreadStart [0x0x7ffda397c34c+44]


## Explicando o Código

Esse código tenta automatizar o jogo Cookie Clicker acessando diretamente o site e adicionando manualmente o cookie `cf_clearance`, que é gerado quando você resolve o CAPTCHA da Cloudflare manualmente. A lógica é parecida com a usada para burlar o CAPTCHA do Google: camuflar o navegador como humano com user-agent, abrir o site previamente e inserir o cookie de autorização. No entanto, neste caso, não funcionou como esperado porque a Cloudflare possui mecanismos de segurança mais avançados que não se limitam apenas à presença do cookie.

O que acontece é que, mesmo com o cookie válido, a Cloudflare pode solicitar uma nova verificação se ela detectar que o navegador é automatizado, o que é comum com Selenium. Isso ocorre porque, além do cookie, ela analisa fatores como movimentação de mouse, tempo de resposta, headers detalhados e principalmente a presença da variável `navigator.webdriver`, que denuncia que o navegador está sendo controlado por um script. Como o código não simula comportamento humano com realismo suficiente, a Cloudflare entende que aquilo ainda pode ser um bot e reapresenta o CAPTCHA.

Existem soluções profissionais para contornar isso, como o serviço Bright Data ([https://brightdata.com/](https://brightdata.com/)), que oferece acesso a proxies rotativos e ferramentas capazes de driblar detecções de bot como as da Cloudflare. No entanto, esse serviço é pago e pode ser caro para uso pessoal ou testes simples, o que fez com que essa alternativa fosse descartada aqui.

![Cookie_erro](CAPTCHA.png)




## Previsão do Tempo - Exemplo


In [ ]:
import requests

api_key = 'f059fb26f708b9fe76ffa9e2a6664e11'

user_input = input("Enter city: ")

weather_data = requests.get(
    f"https://api.openweathermap.org/data/2.5/weather?q={user_input}&units=metric&APPID={api_key}")  # <- "metric"

if weather_data.json()['cod'] == '404':
    print("No City Found")
else:
    weather = weather_data.json()['weather'][0]['main']
    temp = round(weather_data.json()['main']['temp'])

    print(f"The weather in {user_input} is: {weather}")
    print(f"The temperature in {user_input} is: {temp}ºC")  


The weather in rio de janeiro is: Clouds
The temperature in rio de janeiro is: 25ºC


## Explicando o Código

Esse código em Python realiza uma automação simples e poderosa que consulta a previsão do tempo de uma cidade específica usando a API pública do site OpenWeatherMap. A biblioteca `requests` é o principal recurso utilizado, pois permite enviar uma requisição HTTP para o endereço da API, que retorna dados atualizados em formato JSON. O OpenWeatherMap é um serviço online que disponibiliza dados meteorológicos em tempo real de milhares de cidades no mundo todo, e a API key é a chave de acesso individual que autentica o uso da plataforma para cada usuário. Essa automação é útil porque elimina a necessidade de procurar manualmente o clima no navegador, facilitando integrações com sistemas maiores, relatórios automáticos ou aplicações inteligentes. A verificação do código de resposta `'cod' == '404'` garante que o nome da cidade foi reconhecido corretamente, e, em caso positivo, o programa extrai a condição do clima e a temperatura atual, convertida diretamente em Celsius. Esse tipo de automação é um exemplo claro de como integrar APIs com scripts personalizados para coletar, tratar e exibir dados de maneira prática e rápida.


In [ ]:
import requests
import csv
import json

api_key = 'f059fb26f708b9fe76ffa9e2a6664e11'
country_code = input("Enter country code (e.g., BR, US, FR): ").strip().upper()

# Carrega o arquivo de cidades
with open("city.list.json", encoding="utf-8") as file:
    all_cities = json.load(file)

# Filtra cidades do país desejado
cities_in_country = [city for city in all_cities if city["country"] == country_code]

# Prepara o CSV
with open(f"{country_code}_weather.csv", "w", newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["City", "Country", "Weather", "Temperature (C)"])

    for city in cities_in_country[:100]:  # Limite de 100 para evitar bloqueios de API
        city_id = city["id"]
        name = city["name"]

        url = f"https://api.openweathermap.org/data/2.5/weather?id={city_id}&appid={api_key}&units=metric"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            weather = data["weather"][0]["main"]
            temp = round(data["main"]["temp"])
            writer.writerow([name, country_code, weather, temp])
        else:
            print(f"Could not fetch weather for {name}")


## Explicando o Codigo

Esse código automatiza a coleta de dados climáticos de diversas cidades de um país usando a API do OpenWeatherMap e grava as informações em um arquivo CSV. Ele começa pedindo ao usuário o código do país (como BR para Brasil), e em seguida carrega um arquivo JSON chamado `city.list.json`, que contém uma lista com todas as cidades cadastradas na base de dados da OpenWeatherMap, incluindo seus nomes, identificadores e país. O arquivo JSON é essencial nesse processo porque funciona como uma base local estruturada que o código utiliza para filtrar apenas as cidades do país desejado, permitindo chamadas mais precisas e organizadas à API. A API, por sua vez, fornece as informações de temperatura e clima atual de cada cidade a partir de seu identificador único. O motivo pelo qual o código pode dar problema ao tentar consultar todas as cidades do Brasil é que a OpenWeather impõe um limite de requisições por minuto na versão gratuita (normalmente até 60 chamadas), o que significa que fazer centenas ou milhares de chamadas em sequência pode causar bloqueio temporário do IP ou falhas na execução. Além disso, sem intervalos entre as chamadas, o código corre o risco de parar de funcionar no meio da execução, perdendo parte dos dados ou gerando um CSV incompleto. Por isso, embora o script seja útil e eficiente para automatizar relatórios climáticos em escala, ele exige ajustes como pausas (`sleep`) e controle de requisições para funcionar com grandes volumes de cidades.

![Planilha_clima_tempo](dados_clima.png)
![Planilha_clima_tempo_python](planilha_clima_python.png)

# Análise Automatizada de Currículos em PDF

Este notebook automatiza a leitura de currículos em PDF, localizados na pasta `INF1515/curriculos`, e verifica quais candidatos mencionam instituições específicas como:

- **PUC-Rio**
- **Pontifícia Universidade Católica do Rio de Janeiro**

O objetivo é filtrar automaticamente os arquivos relevantes para facilitar a triagem de candidatos.


In [16]:
!pip install pymupdf

In [17]:
import os
import PyPDF2
import fitz  # PyMuPDF

### Leitura automática de currículos em PDF

Esta automação percorre todos os arquivos PDF dentro da pasta `curriculos/` e verifica se o conteúdo menciona "PUC-Rio" ou "Pontifícia Universidade Católica do Rio de Janeiro".

Se encontrar, imprime o nome do arquivo (ou salva numa lista para posterior triagem).


In [12]:
# Caminho da pasta
pasta_curriculos = "C:/Users/pedro/INF1515/curriculos"

# Palavras-chave para buscar nos currículos
palavras_chave = ["PUC-Rio", "Pontifícia Universidade Católica do Rio de Janeiro"]


In [13]:
curriculos_filtrados = []

for nome_arquivo in os.listdir(pasta_curriculos):
    if nome_arquivo.endswith(".pdf"):
        caminho_completo = os.path.join(pasta_curriculos, nome_arquivo)
        doc = fitz.open(caminho_completo)
        texto = ""
        
        for pagina in doc:
            texto += pagina.get_text()
        
        if any(palavra in texto for palavra in palavras_chave):
            curriculos_filtrados.append(nome_arquivo)

# Resultado
print("Currículos que mencionam PUC-Rio ou equivalente:")
for nome in curriculos_filtrados:
    print("-", nome)


Currículos que mencionam PUC-Rio ou equivalente:
- candidato1.pdf
- candidato3.pdf
- candidato5.pdf


## Ultimo passo - salvar nomes candidatos num arquivo excel


In [18]:
import pandas as pd

df = pd.DataFrame(curriculos_filtrados, columns=["Currículos aprovados"])
df.to_excel("curriculos_filtrados.xlsx", index=False)
print("📄 Arquivo salvo como curriculos_filtrados.xlsx")


📄 Arquivo salvo como curriculos_filtrados.xlsx
